In [1]:
import sys
sys.path.append("/home/hugo/projetos-doutorado/Appliance-Energy-Prediction/ae_fts")
sys.path.append("/home/hugo/projetos-doutorado/Appliance-Energy-Prediction/pre_processing")
import AETransformation
import AEWMVFTS
import PreProcessing
from sklearn.metrics import mean_absolute_error
import math
from pyFTS.common import Util
import datetime
import statistics
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Model
from keras.layers import Dense, Input
from keras import regularizers
from sklearn.preprocessing import MinMaxScaler
import pyFTS
from pyFTS.benchmarks import Measures
import statistics
from pyFTS.common.Util import persist_obj, load_obj
from pyFTS.models.multivariate import common, variable, wmvfts
from pyFTS.partitioners import Grid
from pyFTS.common import Membership

# from ae_fts.AETransformation import AutoencoderTransformation
# from ae_fts.AEWMVFTS import AEWeightedMVFTS as AEWMVFTS
# from pre_processing.PreProcessing import PreProcessing

In [2]:
df = pd.read_csv('/home/hugo/projetos-doutorado/Appliance-Energy-Prediction/data/household_power_consumption.csv')
#data = df.drop(labels=['icon','summary','cloudCover', 'House overall [kW]', 'time'], axis=1)
#data = data.loc[0: : 10]
#data.reset_index(inplace=True)
#data.pop('index')

In [3]:
def clean_dataset(df):
    assert isinstance(df, pd.DataFrame) 
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64)

In [4]:
data = pd.read_csv('/home/hugo/projetos-doutorado/Appliance-Energy-Prediction/data/household_power_consumption.csv', sep = ";")
data = data.drop(labels=['Time','Date'], axis=1)
#data = data.loc[0: : 30] # 30 minutes
data.dropna(inplace = True)
data = clean_dataset(data)
data.head()

/home/hugo/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (2,3,4,5,6,7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,4.216,0.418,234.84,18.4,0.0,1.0,17.0
1,5.360,0.436,233.63,23.0,0.0,1.0,16.0
2,5.374,0.498,233.29,23.0,0.0,2.0,17.0
3,5.388,0.502,233.74,23.0,0.0,1.0,17.0
4,3.666,0.528,235.68,15.8,0.0,1.0,17.0


In [5]:
names = ['factor_1']

In [6]:
def sliding_window_AE(data,n_windows,train_size,dim,fs, names):
    ae_wmvfts = AEWMVFTS.AEWeightedMVFTS(n_dimensions = dim,n_part = fs,
                                         endogen_variable='Global_active_power',names=names)
    
    result = {
         "window": [],
         "rmse": [],
         "mape": [],
         "smape": [],
         "mae": []
    }

    order = 1
    tam = len(data)
    n_windows = 30
    windows_length = math.floor(tam / n_windows)
    for ct, ttrain, ttest in Util.sliding_window(data, windows_length, 0.75, inc=1):
        if len(ttest) > 0:
        
            data_train = ttrain.loc[:,'Global_active_power':'Sub_metering_3']
            data_test = ttest.loc[:,'Global_active_power':'Sub_metering_3']
        
        
            print('-' * 20)
            print(f'training window {(ct)}')
            model, ae_reduced_train = ae_wmvfts.run_train_model(data=data_train, epochs=50, n_layers=2, neuron_per_layer=[50,dim], batch=40, opt='adagrad', act='softsign')
            forecast, forecast_self, ae_reduced_test = ae_wmvfts.run_test_model(model=model, data=data_test, epochs=50, n_layers=2,
                                                           neuron_per_layer=[50,dim], batch=40, opt='adagrad', act='aoftsign')
        
        #Teste e cálculo dos erros da previsão
        #appliance_test = ttest['Appliances_t+1'].values
        
        #Teste e cálculo dos erros da previsão - valor use  é Appliances o use Appliances_t+1 --> label igual por causa do model
            print("[{0: %H:%M:%S}]".format(datetime.datetime.now()) + f" getting statistics")
            rmse = Measures.rmse(ae_reduced_test['Global_active_power'].values.tolist(),forecast)
            mape = Measures.mape(ae_reduced_test['Global_active_power'].values.tolist(),forecast)
            smape = Measures.smape(ae_reduced_test['Global_active_power'].values.tolist(),forecast)
        
            forecast = pd.DataFrame(forecast)
            forecast.fillna(forecast.mean(),inplace=True)
            forecast = np.array(forecast).reshape(-1)
            mae = mean_absolute_error(ae_reduced_test['Global_active_power'].values.tolist(), forecast)
        
        # Grafico de comparação Original x Previsto para cada janela
        #fig, ax = plt.subplots(nrows=1, ncols=1, figsize=[15, 3])
        #ax.plot(pca_reduced_test['Appliances'], label='Original')
        #ax.plot(forecast, label='Forecast')
        #handles, labels = ax.get_legend_handles_labels()
        #lgd = ax.legend(handles, labels, loc=2, bbox_to_anchor=(1, 1))
        #plt.show()
        
            result["rmse"].append(rmse)
            result["mape"].append(mape)
            result["smape"].append(smape)
            result["mae"].append(mae)
            result["window"].append(ct)
        
            
    measures = pd.DataFrame(result)
    return measures

In [7]:
dimensions = [3]
fuzzy_sets = [50]

names.append('factor_'+str(2))
names.append('factor_'+str(3))

final_result = {
    "dimensions": [],
    "fuzzy_sets": [],
    "rmse": [],
    "mae": [],
    "mape": [],
    "smape": []
}

n_windows = 30
train_size = 0.75

for dim in dimensions:
    #names.append('factor_'+str(dim))
    print (names)
    for fs in fuzzy_sets:
        measures = sliding_window_AE(data,n_windows,train_size,dim,fs, names)
        final_result["dimensions"].append(dim)
        final_result["fuzzy_sets"].append(fs)
        
        rmse = round(statistics.mean(measures['rmse']),3)
        mape = round(statistics.mean(measures['mape']),3)
        smape = round(statistics.mean(measures['smape']),3)
        mae = round(statistics.mean(measures['mae']),3)
        
        final_result["rmse"].append(rmse)
        final_result["mape"].append(mape)
        final_result["smape"].append(smape)
        final_result["mae"].append(mae)
        
        print(f'Results: {(dim,fs,rmse,mae,mape,smape)}')
        
        
final_measures_ae = pd.DataFrame(final_result) 

print("Statistics AE-WMVFTS (test): ")
final_measures_ae

['factor_1', 'factor_2', 'factor_3']
--------------------
training window 0
Epoch 1/50
1708/1708 [==============================] - 1s 551us/step - loss: 0.0276
Epoch 2/50
1708/1708 [==============================] - 1s 502us/step - loss: 0.0059
Epoch 3/50
1708/1708 [==============================] - 1s 490us/step - loss: 0.0045
Epoch 4/50
1708/1708 [==============================] - 1s 500us/step - loss: 0.0041
Epoch 5/50
1708/1708 [==============================] - 1s 566us/step - loss: 0.0040
Epoch 6/50
1708/1708 [==============================] - 1s 607us/step - loss: 0.0040
Epoch 7/50
1708/1708 [==============================] - 1s 822us/step - loss: 0.0039
Epoch 8/50
1708/1708 [==============================] - 1s 564us/step - loss: 0.0039
Epoch 9/50
1708/1708 [==============================] - 1s 736us/step - loss: 0.0040
Epoch 10/50
1708/1708 [==============================] - 1s 642us/step - loss: 0.0039
Epoch 11/50
1708/1708 [==============================] - 1s 470us/step - 

1708/1708 [==============================] - 1s 787us/step - loss: 0.0039
Epoch 45/50
1708/1708 [==============================] - 1s 523us/step - loss: 0.0038
Epoch 46/50
1708/1708 [==============================] - 1s 456us/step - loss: 0.0038
Epoch 47/50
1708/1708 [==============================] - 1s 513us/step - loss: 0.0038
Epoch 48/50
1708/1708 [==============================] - 1s 522us/step - loss: 0.0038
Epoch 49/50
1708/1708 [==============================] - 1s 513us/step - loss: 0.0039
Epoch 50/50
1708/1708 [==============================] - 1s 517us/step - loss: 0.0039
[ 20:03:50] getting statistics
--------------------
training window 136618
Epoch 1/50
1708/1708 [==============================] - 1s 455us/step - loss: 0.0291
Epoch 2/50
1708/1708 [==============================] - 1s 453us/step - loss: 0.0043
Epoch 3/50
1708/1708 [==============================] - 1s 453us/step - loss: 0.0034
Epoch 4/50
1708/1708 [==============================] - 1s 454us/step - loss: 0.

1708/1708 [==============================] - 1s 495us/step - loss: 0.0031
Epoch 38/50
1708/1708 [==============================] - 1s 532us/step - loss: 0.0032
Epoch 39/50
1708/1708 [==============================] - 1s 529us/step - loss: 0.0031
Epoch 40/50
1708/1708 [==============================] - 1s 532us/step - loss: 0.0031
Epoch 41/50
1708/1708 [==============================] - 1s 533us/step - loss: 0.0031
Epoch 42/50
1708/1708 [==============================] - 1s 499us/step - loss: 0.0031
Epoch 43/50
1708/1708 [==============================] - 1s 520us/step - loss: 0.0031
Epoch 44/50
1708/1708 [==============================] - 1s 534us/step - loss: 0.0031
Epoch 45/50
1708/1708 [==============================] - 1s 517us/step - loss: 0.0031
Epoch 46/50
1708/1708 [==============================] - 1s 520us/step - loss: 0.0031
Epoch 47/50
1708/1708 [==============================] - 1s 517us/step - loss: 0.0032
Epoch 48/50
1708/1708 [==============================] - 1s 523us/

1708/1708 [==============================] - 1s 526us/step - loss: 0.0027
Epoch 31/50
1708/1708 [==============================] - 1s 528us/step - loss: 0.0027
Epoch 32/50
1708/1708 [==============================] - 1s 525us/step - loss: 0.0027
Epoch 33/50
1708/1708 [==============================] - 1s 526us/step - loss: 0.0027
Epoch 34/50
1708/1708 [==============================] - 1s 526us/step - loss: 0.0028
Epoch 35/50
1708/1708 [==============================] - 1s 526us/step - loss: 0.0027
Epoch 36/50
1708/1708 [==============================] - 1s 529us/step - loss: 0.0027
Epoch 37/50
1708/1708 [==============================] - 1s 529us/step - loss: 0.0027
Epoch 38/50
1708/1708 [==============================] - 1s 527us/step - loss: 0.0027
Epoch 39/50
1708/1708 [==============================] - 1s 525us/step - loss: 0.0028
Epoch 40/50
1708/1708 [==============================] - 1s 527us/step - loss: 0.0027
Epoch 41/50
1708/1708 [==============================] - 1s 495us/

1708/1708 [==============================] - 1s 529us/step - loss: 0.0039
Epoch 24/50
1708/1708 [==============================] - 1s 518us/step - loss: 0.0039
Epoch 25/50
1708/1708 [==============================] - 1s 508us/step - loss: 0.0039
Epoch 26/50
1708/1708 [==============================] - 1s 528us/step - loss: 0.0039
Epoch 27/50
1708/1708 [==============================] - 1s 529us/step - loss: 0.0039
Epoch 28/50
1708/1708 [==============================] - 1s 530us/step - loss: 0.0039
Epoch 29/50
1708/1708 [==============================] - 1s 507us/step - loss: 0.0038
Epoch 30/50
1708/1708 [==============================] - 1s 494us/step - loss: 0.0039
Epoch 31/50
1708/1708 [==============================] - 1s 530us/step - loss: 0.0039
Epoch 32/50
1708/1708 [==============================] - 1s 516us/step - loss: 0.0039
Epoch 33/50
1708/1708 [==============================] - 1s 527us/step - loss: 0.0040
Epoch 34/50
1708/1708 [==============================] - 1s 516us/

...

1708/1708 [==============================] - 1s 518us/step - loss: 0.0023
Epoch 24/50
1708/1708 [==============================] - 1s 497us/step - loss: 0.0023
Epoch 25/50
1708/1708 [==============================] - 1s 524us/step - loss: 0.0024
Epoch 26/50
1708/1708 [==============================] - 1s 525us/step - loss: 0.0023
Epoch 27/50
1708/1708 [==============================] - 1s 526us/step - loss: 0.0023
Epoch 28/50
1708/1708 [==============================] - 1s 519us/step - loss: 0.0022
Epoch 29/50
1708/1708 [==============================] - 1s 529us/step - loss: 0.0023
Epoch 30/50
1708/1708 [==============================] - 1s 525us/step - loss: 0.0023
Epoch 31/50
1708/1708 [==============================] - 1s 523us/step - loss: 0.0023
Epoch 32/50
1708/1708 [==============================] - 1s 529us/step - loss: 0.0023
Epoch 33/50
1708/1708 [==============================] - 1s 526us/step - loss: 0.0023
Epoch 34/50
1708/1708 [==============================] - 1s 525us/

1708/1708 [==============================] - 1s 480us/step - loss: 0.0027
Epoch 17/50
1708/1708 [==============================] - 1s 486us/step - loss: 0.0028
Epoch 18/50
1708/1708 [==============================] - 1s 485us/step - loss: 0.0028
Epoch 19/50
1708/1708 [==============================] - 1s 481us/step - loss: 0.0027
Epoch 20/50
1708/1708 [==============================] - 1s 493us/step - loss: 0.0028
Epoch 21/50
1708/1708 [==============================] - 1s 495us/step - loss: 0.0027
Epoch 22/50
1708/1708 [==============================] - 1s 497us/step - loss: 0.0028
Epoch 23/50
1708/1708 [==============================] - 1s 497us/step - loss: 0.0028
Epoch 24/50
1708/1708 [==============================] - 1s 495us/step - loss: 0.0028
Epoch 25/50
1708/1708 [==============================] - 1s 497us/step - loss: 0.0027
Epoch 26/50
1708/1708 [==============================] - 1s 496us/step - loss: 0.0027
Epoch 27/50
1708/1708 [==============================] - 1s 512us/

/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/models/multivariate/mvfts.py:165: RuntimeWarning: invalid value encountered in double_scalars
  ret.append(np.dot(mv,mp.T)/np.nansum(mv))


[ 22:30:57] getting statistics
Results: (3, 50, 0.085, 0.05, 10.863, 4.654)
Statistics AE-WMVFTS (test): 


,dimensions,fuzzy_sets,rmse,mae,mape,smape
0,3,50,0.085,0.05,10.863,4.654


In [ ]:
data = pd.read_csv('/home/hugo/projetos-doutorado/Appliance-Energy-Prediction/data/household_power_consumption.csv', sep = ";")
data = data.drop(labels=['Time','Date'], axis=1)
#data = data.loc[0: : 30] # 30 minutes
data.dropna(inplace = True)
data = clean_dataset(data)
data.head()

In [ ]:
names

In [ ]:
dimensions = [3]
fuzzy_sets = [50]
names = ['factor_1']

final_result = {
    "dimensions": [],
    "fuzzy_sets": [],
    "rmse": [],
    "mae": [],
    "mape": [],
    "smape": []
}

n_windows = 30
train_size = 0.75

for dim in dimensions:
    names.append('factor_'+str(dim))
    print (names)
    for fs in fuzzy_sets:
        measures = sliding_window_AE(data,n_windows,train_size,dim,fs, names)
        final_result["dimensions"].append(dim)
        final_result["fuzzy_sets"].append(fs)
        
        rmse = round(statistics.mean(measures['rmse']),3)
        mape = round(statistics.mean(measures['mape']),3)
        smape = round(statistics.mean(measures['smape']),3)
        mae = round(statistics.mean(measures['mae']),3)
        
        final_result["rmse"].append(rmse)
        final_result["mape"].append(mape)
        final_result["smape"].append(smape)
        final_result["mae"].append(mae)
        
        print(f'Results: {(dim,fs,rmse,mae,mape,smape)}')
        
        
final_measures_ae = pd.DataFrame(final_result) 

print("Statistics AE-WMVFTS (test): ")
final_measures_ae

In [ ]:
final_measures_ae.to_csv (r'ae_wmvfts_dim_fsets_hpc_uci_new.csv', index = False, header=True)